<a href="https://colab.research.google.com/github/DaraSamii/DataDays2021/blob/main/Simple_Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount("/content/drive",force_remount=True)

%cd /content/drive/MyDrive/DataDays2021/

pwd = %pwd
pwd

Mounted at /content/drive
/content/drive/MyDrive/DataDays2021


'/content/drive/MyDrive/DataDays2021'

In [ ]:
!pip install parsivar

In [ ]:
import pandas as pd 
import numpy as np
import os
from helper import utils
from tqdm import tqdm

In [ ]:
import os
data_folder = os.path.join(pwd,"data")
final_clicked = os.path.join(data_folder,"final","final_clicked.csv")
final_products = os.path.join(data_folder,"final","final_products.csv")
mix_model_path = os.path.join(data_folder,"models","mix_model","models","mix_model.pth")
category_classifier_path = os.path.join(data_folder,"models","category_classifier","models","category_classifier.pth")
vocab_path = os.path.join(data_folder,"models","category_classifier","vocab")
category_list_path = os.path.join(data_folder,"models","category_classifier","catagory_list")
queries_test_offline_normalized_path = os.path.join(data_folder, "queries_test_offline_normalized.json")
queries_test_offline_path = os.path.join(data_folder,"final" ,"queries_test_offline.json")
test_queries_floder_path = os.path.join(data_folder,"final","results.csv")

In [ ]:
def Similarity(sent1, sent2):
    try:
        count = 0
        for word1 in sent1.split(" "):
            for word2 in sent2.split(" "):
                if word1 == word2:
                    count+=1
        return count
    except:
        return 0

In [ ]:
products = pd.read_csv(final_products)

In [ ]:
df = products[["category_name","product_name_normalized","sellers_count","availabilty_ratio","COUNT(clicked_merged)","_id"]][products["availabilty_ratio"]!=0.0]

In [ ]:
category_list = list(df["category_name"].unique())

In [ ]:
def query_sorter(query,df,category_list):
    ndf = pd.DataFrame({"category_name":category_list})
    ndf["query"] = query
    ndf["similarity"] = 0
    ndf["similarity"] = ndf.apply(lambda x: Similarity(x["category_name"],x["query"]),axis=1)
    if ndf["similarity"].max() != 0:
        category_list = list(ndf[ndf["similarity"] !=0]["category_name"].values)
        
        pdf  = df[(df["COUNT(clicked_merged)"] != 0) & (df["category_name"].isin(category_list))].copy()
        pdf["query"] = query
        pdf["similarity"] = 0
        pdf["similarity"] = pdf.apply(lambda x: Similarity(x["product_name_normalized"],x["query"]),axis=1)
        if pdf["similarity"].max() != 0:

            return pdf.sort_values(["similarity","COUNT(clicked_merged)"],ascending=[False,False])
        else:
            zdf = df[df["COUNT(clicked_merged)"] == 0][df["category_name"].isin(category_list)].copy()
            zdf["query"] = query
            zdf["similarity"] = 0
            zdf["similarity"] = zdf.apply(lambda x: Similarity(x["product_name_normalized"],x["query"]),axis=1)
  
            return zdf.sort_values(["similarity"],ascending=[False])
    else:
        return 
        '''
        pdf = df[df["COUNT(clicked_merged)"] != 0].copy()
        pdf["query"] = query
        pdf["similarity"] = 0
        pdf["similarity"] = pdf.apply(lambda x: Similarity(x["product_name_normalized"],x["query"]),axis=1)
        if pdf["similarity"].max() != 0:

            return pdf.sort_values(["similarity","COUNT(clicked_merged)"],ascending=[False,False],inplace=True)
        else:
            zdf = df[df["COUNT(clicked_merged)"] == 0].copy()
            zdf["query"] = query
            zdf["similarity"] = 0
            zdf["similarity"] = zdf.apply(lambda x: Similarity(x["product_name_normalized"],x["query"]),axis=1)

            return zdf.sort_values(["similarity"],ascending=False)
        '''

    


In [ ]:
a = query_sorter("تلویزیون",df,category_list)

In [ ]:
list(a.head(10)["_id"])

['mmpmu',
 'mymqd',
 'mdcca',
 'milcn',
 'milcx',
 'mdcqk',
 'mmlnk',
 'mmlrj',
 'mmlnw',
 'mmlnc']

In [ ]:
test_list = []
for i in utils.read_json(queries_test_offline_normalized_path):
    test_list.append(i)

38724it [00:00, 210921.12it/s]


In [ ]:
result = []
N_queries = []
for i in tqdm(range(len(test_list))):
    d = {}
    query = test_list[i]
    d["query"] = query
    a = query_sorter(utils._normalize_text(query), df, category_list)
    if type(a) != type(None):
        if a.shape[0]!=0:
            res_ids = list(a.head(10)["_id"])
            for j in range(10):
                if j < len(res_ids):
                    d[str(j)] = res_ids[j]
                else:
                    d[str(j)] = 'N'
        else:
            for j in range(10):
                d[str(j)] = 'N'
            print("N")
            N_queries.append(query)
    else:
        for j in range(10):
            d[str(j)] = 'N'
        print("N")
        N_queries.append(query)
    result.append(d)

  0%|          | 5/38724 [00:00<1:12:30,  8.90it/s]

N
N
N


  0%|          | 12/38724 [00:01<1:09:56,  9.23it/s]

N


  0%|          | 19/38724 [00:02<1:16:33,  8.43it/s]

...

100%|█████████▉| 38714/38724 [2:57:24<00:03,  3.14it/s]

N


100%|█████████▉| 38721/38724 [2:57:27<00:01,  2.55it/s]

N
N


100%|██████████| 38724/38724 [2:57:27<00:00,  3.64it/s]


In [ ]:
ldf = pd.DataFrame(result)
ldf

,query,0,1,2,3,4,5,6,7,8,9
0,تلویزیون 40 ایتچ,myufm,mnncl,mnmik,mymqr,mtdlc,myukp,moyxh,morvg,mtdoh,myufu
1,قهوه ساز delongi,mmpea,myzfe,myulu,myaog,mmpes,mmpef,mmpxi,mybra,mmhxx,yzery
2,بند فلزی mi band 5,mmmyt,muqog,mmmya,mmmpi,muyjc,mmuxg,mmkng,mmuav,muatw,mmuby
3,قالی زمردمشهد,N,N,N,N,N,N,N,N,N,N
4,پوکو x 3 pro,murxk,muqva,mkbcl,mkbcv,mzhwe,muwxa,mytfw,mzdvq,mdsbw,mdghk
...,...,...,...,...,...,...,...,...,...,...,...
38719,نوت 10,mmxlt,mlnzr,mjvfi,mmkcl,mgnsb,mhwnr,movyg,mmqvy,muzqp,mqnrb
38720,اسباب بازی بتمت,yikye,mhazg,mmmis,mmmic,mmmir,mybgh,mksca,mmuke,muefs,mmbdx
38721,کراسفیت,N,N,N,N,N,N,N,N,N,N
38722,mate 30 pro,N,N,N,N,N,N,N,N,N,N


In [ ]:
ldf.to_csv(os.path.join(data_folder, "final","simple_way.csv"),index=False,header=False)

In [ ]:
len(N_queries)

9982

In [ ]:
df[df["_id"] == "myzfe"]

,category_name,product_name_normalized,sellers_count,availabilty_ratio,COUNT(clicked_merged),_id
1593807,سایر انواع و برندهای قهوه ساز,قهوه bk ccm 770 ساز cm cm 770 قهوه cm 770 r جو...,15,0.133333,12.0,myzfe


In [ ]:
!pip install git+https://github.com/RoboEpics/roboepics-client.git@d50db61a92ab14f5e95c87679b681a0836a3f6e5
from roboepics_client.roboepics_client import RoboEpicsClient

problem_id = 4  # This value is set according to your user. Do not change it please.
problem_enter_id = 1574  # This value is set according to your user. Do not change it please.

roboepics_client = RoboEpicsClient(problem_id, problem_enter_id)

  Cloning https://github.com/RoboEpics/roboepics-client.git (to revision d50db61a92ab14f5e95c87679b681a0836a3f6e5) to /tmp/pip-req-build-m69ph_ez
  Running command git clone -q https://github.com/RoboEpics/roboepics-client.git /tmp/pip-req-build-m69ph_ez
  Running command git rev-parse -q --verify 'sha^d50db61a92ab14f5e95c87679b681a0836a3f6e5'
  Running command git fetch -q https://github.com/RoboEpics/roboepics-client.git d50db61a92ab14f5e95c87679b681a0836a3f6e5
Open this URL and confirm your login: https://fusion.roboepics.com/oauth2/device?client_id=7126a051-baea-4fe1-bdf8-fde2fdb31f97&user_code=ZR68GP


KeyboardInterrupt: ignored

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(os.path.join(data_folder, "final","simple_way.csv"))

In [ ]:
ndf = pd.read_csv(os.path.join(data_folder, "final","output.csv"))

In [ ]:
ndf

,تلویزیون 40 ایتچ,N,N.1,N.2,N.3,N.4,N.5,N.6,N.7,N.8,N.9
0,قهوه سازDelongi,N,N,N,N,N,N,N,N,N,N
1,بند فلزی mi band 5,N,N,N,N,N,N,N,N,N,N
2,قالی زمردمشهد,N,N,N,N,N,N,N,N,N,N
3,پوکوx3 pro,N,N,N,N,N,N,N,N,N,N
4,Xiaomi redmi note 9s,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...
38718,نوت ۱۰+,N,N,N,N,N,N,N,N,N,N
38719,اسباب‌بازی‌بتمت,N,N,N,N,N,N,N,N,N,N
38720,کراسفیت,N,N,N,N,N,N,N,N,N,N
38721,mate30pro,N,N,N,N,N,N,N,N,N,N


In [ ]:
df.head(50)

,تلویزیون 40 ایتچ,myufm,mnncl,mnmik,mymqr,mtdlc,myukp,moyxh,morvg,mtdoh,myufu
0,قهوه ساز delongi,mmpea,myzfe,myulu,myaog,mmpes,mmpef,mmpxi,mybra,mmhxx,yzery
1,بند فلزی mi band 5,mmmyt,muqog,mmmya,mmmpi,muyjc,mmuxg,mmkng,mmuav,muatw,mmuby
2,قالی زمردمشهد,N,N,N,N,N,N,N,N,N,N
3,پوکو x 3 pro,murxk,muqva,mkbcl,mkbcv,mzhwe,muwxa,mytfw,mzdvq,mdsbw,mdghk
4,xiaomi redmi note 9 s,N,N,N,N,N,N,N,N,N,N
5,40 d,N,N,N,N,N,N,N,N,N,N
6,لباس انا,maczh,mtuwq,mfuvc,urcsi,myfzh,myfdu,myfdm,myfdt,mkaty,myfdk
7,سنگ چشم ببر,molbd,molbk,molbu,molbb,molbt,molba,molbz,molbf,moltf,molbc
8,گلگیر جلو پژو 405 شرکتی,mgnlv,mocmu,mnyry,mnyra,mnyrz,mnyrk,mnyru,mnyrt,mncfs,mncfr
9,فیس برواش,mzuiv,mzmxu,mtdmp,mtdmf,mtdmn,mtptc,mtdmh,mdujb,mtqvi,mtdmr
